# Part 1

Suddenly, whirling in the distance, you notice what looks like a massive, pixelated hurricane: a deadly spinlock. This spinlock isn't just consuming computing power, but memory, too; vast, digital mountains are being ripped from the ground and consumed by the vortex.

If you don't move quickly, fixing that printer will be the least of your problems.

This spinlock's algorithm is simple but efficient, quickly consuming everything in its path. It starts with a circular buffer containing only the value 0, which it marks as the current position. It then steps forward through the circular buffer some number of steps (your puzzle input) before inserting the first new value, 1, after the value it stopped on. The inserted value becomes the current position. Then, it steps forward from there the same number of steps, and wherever it stops, inserts after it the second new value, 2, and uses that as the new current position again.

It repeats this process of stepping forward, inserting a new value, and using the location of the inserted value as the new current position a total of 2017 times, inserting 2017 as its final operation, and ending with a total of 2018 values (including 0) in the circular buffer.

For example, if the spinlock were to step 3 times per insert, the circular buffer would begin to evolve like this (using parentheses to mark the current position after each iteration of the algorithm):

- `(0)`, the initial state before any insertions.
- `0 (1)`: the spinlock steps forward three times `(0, 0, 0)`, and then inserts the first value, 1, after it. 1 becomes the current position.
- `0 (2) 1`: the spinlock steps forward three times `(0, 1, 0)`, and then inserts the second value, 2, after it. 2 becomes the current position.
- `0  2 (3) 1`: the spinlock steps forward three times `(1, 0, 2)`, and then inserts the third value, 3, after it. 3 becomes the current position.

And so on:

```
0  2 (4) 3  1
0 (5) 2  4  3  1
0  5  2  4  3 (6) 1
0  5 (7) 2  4  3  6  1
0  5  7  2  4  3 (8) 6  1
0 (9) 5  7  2  4  3  8  6  1
```

Eventually, after 2017 insertions, the section of the circular buffer near the last insertion looks like this:

```
1512  1134  151 (2017) 638  1513  851
```

Perhaps, if you can identify the value that will ultimately be after the last value written `(2017)`, you can short-circuit the spinlock. In this example, that would be `638`.

What is the value after `2017` in your completed circular buffer?

Your puzzle input is `386`.

In [1]:
from collections import deque

In [2]:
def track_spinlock(step_size, max_val):
    pos = 0
    buf = deque([0])
    
    for i in range(1, max_val + 1):
        # figure out insertion position
        pos = (pos + step_size) % len(buf) + 1
        buf.insert(pos, i)
    
    return buf, pos

In [3]:
def track_spinlock_l(step_size, max_val):
    pos = 0
    buf = [0]
    
    for i in range(1, max_val + 1):
        # figure out insertion position
        pos = (pos + step_size) % len(buf) + 1
        buf.insert(pos, i)
    
    return buf, pos

In [4]:
%load_ext cython

In [5]:
%%cython
from cpython cimport array
import array

cpdef object cy_track_spinlock(unsigned int step_size, unsigned int max_val):
    cdef unsigned int pos = 0
    cdef unsigned int buf_len = 1
    cdef array.array buf_arr = array.array('I', [0] * (max_val + 1))
    cdef unsigned int[:] buf = buf_arr
    cdef unsigned int i
    
    for i in range(1, max_val + 1):
        pos = (pos + step_size) % buf_len + 1
        buf[pos + 1:buf_len + 1] = buf[pos:buf_len]
        buf[pos] = i
        buf_len += 1
    
    return buf_arr, pos

In [6]:
rbuf, rpos = track_spinlock(3, 2017)
assert rbuf[rpos + 1] == 638, rbuf[rpos + 1]

In [7]:
rbuf, rpos = cy_track_spinlock(3, 2017)
assert rbuf[rpos + 1] == 638, rbuf[rpos + 1]

In [8]:
rbuf, rpos = track_spinlock(386, 2017)
rbuf[rpos + 1]

419

In [9]:
rbuf, rpos = track_spinlock_l(386, 2017)
rbuf[rpos + 1]

419

In [10]:
rbuf, rpos = cy_track_spinlock(386, 2017)
rbuf[rpos + 1]

419

In [17]:
%timeit track_spinlock(386, 2017)

2.09 ms ± 95.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
%timeit track_spinlock_l(386, 2017)

1.6 ms ± 76.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [33]:
%timeit cy_track_spinlock(386, 2017)

888 µs ± 84.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Part 2

The spinlock does not short-circuit. Instead, it gets more angry. At least, you assume that's what happened; it's spinning significantly faster than it was a moment ago.

You have good news and bad news.

The good news is that you have improved calculations for how to stop the spinlock. They indicate that you actually need to identify the value after 0 in the current state of the circular buffer.

The bad news is that while you were determining this, the spinlock has just finished inserting its fifty millionth value (`50000000`).

What is the value after `0` the moment `50000000` is inserted?

Your puzzle input is still `386`.

(In this one I don't care about any values except those inserted at position 1.
We can process where all values _would_ be inserted and pretend to grow the size
of the array as we do, but only store values inserted at position 1.
This saves a huge amount of time because no insertions are happening.
(Insertions can be slow with large arrays because they have to move data around.))

In [18]:
%%cython
from cpython cimport array
import array

cpdef unsigned int cy_track_spinlock2(unsigned int step_size, unsigned int max_val):
    cdef unsigned int pos = 0
    cdef unsigned int buf_len = 1
    cdef unsigned int pos1_val
    cdef unsigned int i
    
    for i in range(1, max_val + 1):
        pos = (pos + step_size) % buf_len + 1
        if pos == 1:
            pos1_val = i
        buf_len += 1
    
    return pos1_val

In [19]:
%%time
result = cy_track_spinlock2(386, 50000000)
print(result)

46038988
CPU times: user 366 ms, sys: 3.67 ms, total: 370 ms
Wall time: 371 ms
